<h1 style="text-align: center;">Programa de Alocação de Voluntários</h1>
<p style="text-align: center; font-size:12px;">Leonardo Gama Assumpção - UFRJ - 30/03/2020</p>

Em tempos recentes, devido à pandemia, o Restaurante Universitário passou a oferecer as refeições a um público mais restrito, na forma de quentinhas entregues no Hospital Universitário, no Alojamento Estudantil e na Vila Residencial. Em particular para este último público, 200 a 250 quentinhas têm sido entregues duas vezes ao dia (para almoço e janta) na Associação de Moradores da Vila Residencial.

Com isso, estudantes universitários da Vila Residencial têm se oferecido para ajudar na distribuição das quentinhas que chegam a cada turno, sendo oportuno, portanto, um sistema que permita a alocação de voluntários para dividir nossos esforços de modo a facilitar a participação de todos segundo a disponibilidade de cada um, evitando possíveis sobrecargas.

No que segue, apresentamos a modelagem do problema como um programa de otimização linear com variáveis binárias.

In [1]:
import numpy as np
from sys import path
path.append("../src")
import schedule as sc

cp = sc.cp  # <= (import cvxpy as cp)
# from imp import reload ; reload(sc)

In [2]:
# Arquivos de Dados:
table_fname = sc.default_data_file
volunteers_fname = sc.default_volunteers_file
print('# DADOS:        "{}"\n# VOLUNTÁRIOS:  "{}"'.format(table_fname, volunteers_fname))

# DADOS:        "../data/tabela.txt"
# VOLUNTÁRIOS:  "../data/voluntarios.txt"


&nbsp;
## Dados
&nbsp;

In [3]:
names = sc.read_data(volunteers_fname).split("\n")
sc.print_enumeration(names)

01: Amanda Aparecida Cordeiro Lucio
02: Andria da Silva Oliveira Roza
03: Annatercia Gomes Pinheiro
04: Beatriz Soares Bernardo da Silva
05: Bianca Bernardi Duarte
06: Breno Henrique Oliveira Santos
07: Charlie Vargas Sarmiento
08: Daisy Scarlett Dienett Zubiate Augustin
09: Danilo dos Santos Alves Bezerra
10: Fabiana Pifano Pinto Vilar
11: Felipe de Oliveira Miguel
12: Ferdinando Ribeiro Freire
13: Gabriel Picanço Costa
14: Hariom Nunes Choudhury
15: Hélen Barcellos da Silva Martins
16: Isadora Pinheiro Duarte
17: Jonatã Arcas Silva
18: Joyce Alves do Nascimento
19: Khaian Laybinitz
20: Leon Loureiro Gadelha Angelo Silvestre
21: Leonardo Gama Assumpção
22: Letícia Braga Portes Alves Rentz
23: Lorena Alves Damacena Basílio
24: Lucas Dias Iglezias Castanheira
25: Markus Vinicius Gallo Honório Cunha
26: Matheus Araújo de Oliveira
27: Miguel Angel Pineda Reyes
28: Otávio Lucati Junior
29: Thiago Costa Pereira


In [4]:
sc.print_enumeration(sc.read_data(table_fname).split("\n"))

01: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
02: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
03: 0	.	.	.	.	O	O	.	.	O	O	.	.	.	.
04: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
05: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
06: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
07: 0	O	O	O	O	.	.	.	.	.	.	.	.	O	O
08: 0	.	.	.	.	O	O	.	.	O	O	.	.	.	.
09: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
10: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
11: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
12: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
13: 0	OO	X	X	X	X	X	X	X	X	X	X	X	O	X
14: 0	.	.	O	O	O	O	O	O	.	.	OO	OO	OO	OO
15: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
16: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
17: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
18: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
19: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
20: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
21: 0	OO	OO	X	X	X	X	X	X	X	X	X	X	X	X
22: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
23: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
24: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
25: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
26: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
27: 0	O	O	.	.	O	O	.	.	O	O	.	.	O	O
28: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
29: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.


# Modelo
A proposta do programa é oferecer uma alocação para os voluntários por um período fixo, de modo a evitar que pessoas fiquem sobrecarregadas e levar em conta as preferências de dias e horários de cada pessoa na tomada de decisão.

Tal problema pode ser modelado como um programa de otimização linear com variáveis binárias, como veremos a seguir. Comecemos fixando algumas

## Definições
- $M := \texttt{min-staff}$: número mínimo de voluntários por turno (atualmente, $M=7$)
- $\texttt{i}$: número (código) da pessoa. vai de $1$ a $N$ (atualmente, $N=27$)
- $\texttt{j}$: número (código) do turno. vai de $1$ a $T$ (atualmente, $T=14$)
- $A_{ij}$: indicadora ($0$ ou $1$) da pessoa $\texttt{i}$ estar disponível no turno $\texttt{j}$
- $P_{ij}$: indicadora ($0$ ou $1$) da pessoa $\texttt{i}$ ter preferência pelo turno $\texttt{j}$
- $w_i$: número (inteiro) de turnos contribuídos pela pessoa $\texttt{i}$ na semana anterior.

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**Variável de decisão do nosso problema:**
- $Z_{ij}$: indicadora da pessoa $\texttt{i}$ ser alocada para colaborar no turno $\texttt{j}$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**Variável auxiliar** (para algumas contas ficarem mais compreensíveis)**:**
- $L_i = \texttt{load}\left[\texttt{i}\right]$: número de turnos alocados para a pessoa $\texttt{i}$.

Para ilustrar a definição, observemos que $L_i$ é exatamente a soma da linha $\texttt{i}$ de $Z$:
$$L_i = \sum_{j=1}^T Z_{ij}$$

## Função de Custo
Temos diversos objetivos concorrentes:
<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**1. Minimizar a carga máxima por pessoa:**

Sendo possível alocar $M$ pessoas por turno com cada uma das $N$ pessoas contribuindo com no máximo $4$ turnos, por exemplo, tal cenário será preferível a qualquer outro com uma ou mais pessoas sobrecarregadas com $5$ ou $6$ turnos na semana.
Uma expressão matemática para esse custo seria
$$\max_i \left| \sum_{j=1}^T Z_{ij} \right| = \max_i |L_i| = \max_i L_i = \left\lVert L \right\rVert_\infty.$$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**2. Evitar disparidades no volume de trabalho:**

Resguardando-se que o sistema consiga alocar o mínimo de $M$ pessoas por turno, preferimos um cenário no qual todos contribuem de modo semelhante, em contraste a outro cenário onde alguns trabalham $4$ turnos e outros trabalham $2$.
Assim, se $\overline{L}$ é a média dos $L_i$, ajustamos uma constante $\beta$ de normalização e escrevemos:
$$\beta \sum_{i=1}^N \left| L_i - \overline{L} \right| = \beta \, \left\lVert L-\overline{L}\right\rVert_1$$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

Assim, se $\mathbb{1} := (1, \ldots, 1)$, como $L = Z \cdot \mathbb{1}$, nossa função de custo é dada por
$$f(Z) = \varphi(L) = \left\lVert L \right\rVert_\infty + \beta \, \left\lVert L-\overline{L}\right\rVert_1.$$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**Outras ideias a considerar:**
(1) penalizar soluções com pessoas ajudando em apenas um turno do dia (tendo em vista a hipótese das transições complicarem a logística entre almoço e janta); (2) penalizar a contribuição de uma mesma pessoa em dias consecutivos.

## Restrições

A restrição fundamental é atingir o mínimo de $M$ voluntários por turno:
$$\forall \, j: \; \sum_i Z_{ij} = M.$$

Além disso, gostaríamos de exigir que $Z \preceq A$, ou seja, que cada entrada $Z_{ij}$ seja igual ou inferior a $A_{ij}$, de modo a evitar a alocação de pessoas em turnos que não foram marcados como disponíveis: cada $(i, j)$ com $A_{ij} = 0$ impõe $Z_{ij} = 0$.

Contudo, tendo em vista que a planilha ainda está sendo preenchida, tal restrição foi incorporada à função de custo como penalização &mdash; fixamos um valor $\alpha$ de custo para cada entrada $(i,j)$ com $Z_{ij}=1$ e $A_{ij}=0$:
$$\alpha\sum_{i, j} \max(Z_{ij} - A_{ij}, \, 0) = \alpha\sum_{i, j} \left[ Z_{ij} - A_{ij} \right]_+.$$

&nbsp;
## Soluções
&nbsp;

A seguir, chamamos a função `main()` do arquivo `schedule.py`, que monta, resolve e exibe a solução do problema para diversos valores de $M$.

&nbsp;
### Caso M=5
&nbsp;

In [5]:
results = sc.main(table_fname, min_staff=5, verbose=0)
sc.print_enumeration(results["Z_array"])

Using license file /opt/gurobi901/gurobi.lic
Academic license - for non-commercial use only
01: [0 0 1 0 0 0 0 0 0 0 0 1 0 0]
02: [1 0 1 0 0 0 0 0 0 0 0 0 0 0]
03: [0 0 0 0 0 1 0 0 1 1 0 0 0 0]
04: [0 0 0 0 1 0 1 0 0 0 0 0 0 1]
05: [0 0 0 1 0 1 0 0 0 0 1 0 0 0]
06: [1 0 0 0 0 0 0 0 0 0 0 0 0 1]
07: [0 1 0 0 0 0 0 0 0 0 0 0 1 1]
08: [0 0 0 0 0 1 0 0 1 1 0 0 0 0]
09: [0 0 0 0 0 0 1 1 0 0 0 0 0 0]
10: [0 0 1 0 1 0 0 0 0 0 0 0 0 0]
11: [0 1 0 0 0 0 0 0 0 0 0 0 1 0]
12: [0 0 1 0 0 1 0 1 0 0 0 0 0 0]
13: [1 0 0 0 0 1 0 0 0 0 0 0 1 0]
14: [0 0 0 1 1 0 1 0 0 0 0 0 0 0]
15: [0 0 0 0 0 0 0 0 1 1 0 0 0 0]
16: [0 0 0 0 0 0 0 0 0 1 0 0 0 1]
17: [0 0 0 0 0 0 1 0 0 0 0 1 0 0]
18: [0 0 1 1 0 0 0 0 0 0 0 0 0 0]
19: [0 1 0 0 0 0 0 0 0 0 1 0 0 0]
20: [0 0 0 0 1 0 0 0 0 0 0 0 1 0]
21: [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
22: [0 0 0 0 0 0 0 1 0 0 1 0 0 0]
23: [1 0 0 0 0 0 0 0 1 0 0 1 0 0]
24: [0 0 0 0 0 0 0 1 0 0 1 1 0 0]
25: [0 0 0 0 0 0 0 1 0 0 0 1 0 0]
26: [0 0 0 0 0 0 1 0 0 0 1 0 0 0]
27: [0 1 0 0 0 0 0 0 1 0

&nbsp;
### Caso M=6
&nbsp;

In [6]:
# Adicionando um limite de tempo (que nem deve ser necessário, mas enfim).
gurobi_options = {"verbose": True, "TimeLimit": 30}
results = sc.main(table_fname, min_staff=6, verbose=2, solver_options=gurobi_options)

# Programa de otimização linear com variáveis inteiras (MIP)

Dados de Entrada:

[Array: AVAILABLE]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 1 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


&nbsp;
### Caso M=7
&nbsp;

In [7]:
results = sc.main(table_fname, min_staff=7, verbose=2, solver_options=gurobi_options)

# Programa de otimização linear com variáveis inteiras (MIP)

Dados de Entrada:

[Array: AVAILABLE]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 1 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [8]:
print(*results.items(), sep="\n\n")

('Z', Variable((29, 14), boolean=True))

('Z_array', array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 

&nbsp;
<header>
    <h3>To Be Continued...</h3>
    <p style="text-align:right;">(aguardando a galera preencher a planilha $\ddot\smile$)</p>
</header>